### Mapping Changes to the Unemployment Rate using Plotly

*Brian Dew, Domestic Program Intern, CEPR*<br>
*June 17, 2017*

In [1]:
# Import libraries and sign in to plotly
import pandas as pd
import config   # File that contains api key
import plotly.plotly as py
from plotly.graph_objs import *
py.sign_in('bdew', config.plotly_key)

In [2]:
# CSV file contains city names, their BLS codes (Metropolitan 
# MTAs/NECTAs only), and their latitude and longitude
MSA = pd.read_csv('MSA.csv')

# BLS flat file contains the unemployment rate by month since 2015
url = 'https://download.bls.gov/pub/time.series/la/la.data.0.CurrentU15-19'
df = pd.read_table(url, sep='\s+')

In [3]:
# Take only the unemployment data for MSAs and monthly series 
# (M13 is annual average)
df = pd.DataFrame(df[df['series_id'].isin(MSA['fullcode']) 
                 & ~df['period'].isin(['M13'])])

# Clean up the date to convert it to pandas datetime
df.loc[:,'month'] = df['period'].replace(regex=True, 
                                         to_replace=r'M', 
                                         value=r'')
df.loc[:,'day'] = 1
df.loc[:,'date'] = pd.to_datetime(df[['year', 'month', 'day']])
df = df.set_index(['series_id','date'])

# Calculate one year change in unemployment rate for each MSA
df.loc[:,'value_ch'] = df['value'].diff(periods=12)
#df = df.dropna()

# Keep only the latest value for each MSA
df = df.reset_index().groupby(['series_id']).tail(1)
df.rename(columns={'series_id':'fullcode'}, inplace=True)

# Take the date for use in chart title
date = df['date'].iloc[0].strftime('%B %Y')

# Combine the two datasets and show 2 rows to verify 
unemp_list = df.merge(MSA, on='fullcode')

# Shorten MSA names for chart labels
unemp_list.loc[:,'MSA'] = unemp_list['area_code'].str.replace('Metropolitan Statistical Area', 'MSA')

In [4]:
# From the change in unemployment rate, mark increases red, decreases green,
#   no change is gray. The size of the circle to draw is the absolute value 
#   of the change times 8. 

for index, row in unemp_list.iterrows():
    if row['value_ch'] > 0:
        unemp_list.loc[index,'color'] = '#FF0000'
        unemp_list.loc[index,'size'] = abs(row['value_ch'])*10+2
        unemp_list.loc[index,'sym'] = '&#9650;{}'.format(unemp_list.loc[index,'value_ch']) # Up Arrow
    if row['value_ch'] < 0:
        unemp_list.loc[index,'color'] = '#32CD32'
        unemp_list.loc[index,'size'] = abs(row['value_ch'])*10+2
        unemp_list.loc[index, 'sym'] = '&#9660;{}'.format(unemp_list.loc[index,'value_ch']) # Down Arrow
    if row['value_ch'] == 0:
        unemp_list.loc[index,'color'] = '#696969'
        unemp_list.loc[index,'size'] = 3   # Just to mark location
        unemp_list.loc[index, 'sym'] = 'unchanged'
    # Create a text column for plotly hover labels
    unemp_list.loc[index,'text'] = '{}<br>{} Rate: {}% ({})'.format(
        unemp_list.loc[index,'MSA'], 
        date,
        unemp_list.loc[index,'value'],
        unemp_list.loc[index,'sym'])

#### Plotly code

In [5]:
trace1 = {
  "geo": "geo", 
  "hoverinfo": "text", 
  "lat": unemp_list['latitude'], 
  "lon": unemp_list['longitude'], 
  "marker": {
    "color": unemp_list['color'], 
    "line": {
      "color": "#202020", 
      "width": 0.8
    }, 
    "opacity": 0.5, 
    "size": unemp_list['size'], 
    "symbol": "circle"
  }, 
  "mode": "markers", 
  "name": "Unemployment Rate Change by Metro Area", 
  "opacity": 1, 
  "showlegend": True, 
  "text": unemp_list['text'], 
  "type": "scattergeo", 
  "visible": True
}
data = Data([trace1])
layout = {
  "annotations": [
    {
      "x": 0.05, 
      "y": 0.925, 
      "align": "left", 
      "font": {"size": 14}, 
      "showarrow": False, 
      "text": "<b>Change in Unemployment Rate by Metropolitan Statistical Area (MSA)</b><br><i>Change over level in same month of previous year, as of {}</i>".format(date), 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 0.05, 
      "y": 0.0725, 
      "align": "left", 
      "showarrow": False, 
      "text": "Hover to see details. Green is decrease in unemployment rate, red is increase. Circle size shows amount of change.  <br><b>Source:</b> U.S. Bureau of Labor Statistics, Local Area Unemployment Statistics", 
      "xref": "x", 
      "yref": "y"
    }
  ], 
  "autosize": True, 
  "dragmode": "zoom", 
  "geo": {
    "bgcolor": "#fff", 
    "countrycolor": "#808080", 
    "countrywidth": 0.7, 
    "domain": {
      "x": [0, 1.1], 
      "y": [0, 1.1]
    }, 
    "lataxis": {
      "dtick": 10, 
      "range": [20, 80], 
      "showgrid": False, 
      "tick0": 20
    }, 
    "lonaxis": {
      "dtick": 30, 
      "range": [-180, -50], 
      "showgrid": False, 
      "tick0": -180
    }, 
    "projection": {
      "scale": 1.03, 
      "type": "albers usa"
    }, 
    "resolution": 50, 
    "scope": "usa", 
    "showcountries": True, 
    "showlakes": False, 
    "showland": False, 
    "showrivers": False, 
    "showsubunits": True, 
    "subunitcolor": "#808080", 
    "subunitwidth": 0.7
  }, 
  "hidesources": False, 
  "hovermode": "closest", 
  "paper_bgcolor": "#fff", 
  "separators": ".,", 
  "showlegend": False, 
  "smith": False, 
  "margin": {
    "r": 0, 
    "t": 0, 
    "b": 0, 
    "l": 0
  },
}
fig = Figure(data=data, layout=layout)
plot_url = py.plot(fig, filename='MSA_Map')

#### Generate an html table of results

In [6]:
# generate html table with data
table = unemp_list[['MSA', 'value', 'value_ch']]
table.columns = ['Metropolitan Statistical Area (MSA) Name', 'Unemp. rate', 'One-year change']
table = table.sort_values('One-year change') 
# Sortable css class allows for js script to sort each column
# https://www.kryogenix.org/code/browser/sorttable/
table.to_html('C:/Working/bdecon.github.io/Dash/unemp_table.html', border=0, 
              index=False, classes='sortable')